In [75]:
# TARGET_LOCATION = 'Plymouth, MA'
TARGET_LOCATION = 'Andover, MA'

from model_lib import RedfinModel

In [76]:
redfin = RedfinModel(TARGET_LOCATION, {
  'style': ['SINGLE_FAMILY', 'TOWNHOUSE'],
  'beds': [3, 4, 5]
})
train_df = redfin.fetch_data('sold')
#
train_df.head()

Using cached data for Andover, MA sold as of 2024-01-15
filtering column: style allowed_values: ['SINGLE_FAMILY', 'TOWNHOUSE']
filtering column: beds allowed_values: [3, 4, 5]
Filtered data shape: (153, 29) (from (238, 29))


,property_url,mls,mls_id,status,style,street,unit,city,state,zip_code,...,last_sold_date,lot_sqft,price_per_sqft,latitude,longitude,stories,hoa_fee,parking_garage,primary_photo,alt_photos
2,https://www.realtor.com/realestateandhomes-det...,BSMA,73173904.0,SOLD,SINGLE_FAMILY,5 Courtney Ln,NaN,Andover,MA,1810,...,2024-01-05,15246.0,300,42.613500,-71.125306,NaN,75.0,2.0,http://ap.rdcpix.com/0cb2af0a2ad06741d96c52731...,http://ap.rdcpix.com/0cb2af0a2ad06741d96c52731...
4,https://www.realtor.com/realestateandhomes-det...,BSMA,73180578.0,SOLD,SINGLE_FAMILY,22 Powers Rd,NaN,Andover,MA,1810,...,2024-01-04,31799.0,282,42.616265,-71.141937,NaN,0.0,3.0,http://ap.rdcpix.com/fcba90392c92525376b8e6872...,http://ap.rdcpix.com/fcba90392c92525376b8e6872...
5,https://www.realtor.com/realestateandhomes-det...,BSMA,73170147.0,SOLD,SINGLE_FAMILY,3 Larchmont Cir,NaN,Andover,MA,1810,...,2023-12-29,30492.0,458,42.655440,-71.224991,NaN,0.0,2.0,http://ap.rdcpix.com/c504e697b99880662c687143a...,http://ap.rdcpix.com/c504e697b99880662c687143a...
6,https://www.realtor.com/realestateandhomes-det...,BSMA,73172659.0,SOLD,SINGLE_FAMILY,28 Sagamore Dr,NaN,Andover,MA,1810,...,2023-12-29,43560.0,303,42.622877,-71.113432,NaN,0.0,2.0,http://ap.rdcpix.com/bcd37d8b5c5223570fa4e682a...,http://ap.rdcpix.com/bcd37d8b5c5223570fa4e682a...
9,https://www.realtor.com/realestateandhomes-det...,BSMA,73175289.0,SOLD,SINGLE_FAMILY,8 Burton Farm Dr,NaN,Andover,MA,1810,...,2023-12-29,31799.0,505,42.665803,-71.133468,NaN,0.0,2.0,http://ap.rdcpix.com/92f28bb1233f82ae65b438bf5...,http://ap.rdcpix.com/92f28bb1233f82ae65b438bf5...


In [77]:
train_df.columns.values
# one hot encode columns

array(['property_url', 'mls', 'mls_id', 'status', 'style', 'street',
       'unit', 'city', 'state', 'zip_code', 'beds', 'full_baths',
       'half_baths', 'sqft', 'year_built', 'days_on_mls', 'list_price',
       'list_date', 'sold_price', 'last_sold_date', 'lot_sqft',
       'price_per_sqft', 'latitude', 'longitude', 'stories', 'hoa_fee',
       'parking_garage', 'primary_photo', 'alt_photos'], dtype=object)

In [78]:
redfin.train_from_raw(train_df, train_df[RedfinModel.TARGET_COLUMN])


GradientBoostingRegressor()

In [79]:
mean_score = redfin.show_cross_validation(train_df.drop(RedfinModel.TARGET_COLUMN, axis=1), train_df[RedfinModel.TARGET_COLUMN])


Cross-validation scores: [0.71177741 0.61610479 0.46096984 0.66383585 0.45615923]
Average cross-validation score: 0.58


In [80]:
test_df = redfin.fetch_data('for_sale')
# test_df = redfin.filter_data(test_df)
result_df = redfin.predict(test_df)

Using cached data for Andover, MA for_sale as of 2024-01-15
filtering column: style allowed_values: ['SINGLE_FAMILY', 'TOWNHOUSE']
filtering column: beds allowed_values: [3, 4, 5]
Filtered data shape: (8, 29) (from (13, 29))
Dropping columns: []
Adding columns: ['city=Ballardvale' 'city=In Town' 'city=Shawsheen Village'
 'city=South Andover' 'city=West Andover']
Predicted 8 values


In [81]:

# result_df.sort_values(by=['diff_percent'], ascending=False).head(10)


In [82]:
print(redfin.trained_columns)
# remove exponent formatting
result_df[RedfinModel.OUTPUT_COLUMNS].sort_values(by=['diff_percent'], ascending=False).head(25)


['beds' 'full_baths' 'half_baths' 'hoa_fee' 'lot_sqft' 'parking_garage'
 'sqft' 'stories' 'year_built' 'city=Andover' 'city=Ballardvale'
 'city=In Town' 'city=Shawsheen Village' 'city=South Andover'
 'city=West Andover' 'state=MA' 'style=SINGLE_FAMILY']


,readable_address,style,beds,full_baths,list_price,predicted,diff,diff_percent,latitude,longitude,primary_photo,property_url
0,"124 Lovejoy Rd, Andover, MA",SINGLE_FAMILY,4.0,4.0,999900,1.770672e+06,770771.835896,77.084892,42.646538,-71.179352,http://ap.rdcpix.com/ac67958e0fea7a9ea2824ad54...,https://www.realtor.com/realestateandhomes-det...
7,"430 Lowell St, Andover, MA",SINGLE_FAMILY,3.0,3.0,969000,1.165967e+06,196966.961751,20.326828,42.642851,-71.203751,http://ap.rdcpix.com/e26c5e6e517f6cc600a9ec020...,https://www.realtor.com/realestateandhomes-det...
2,"285 N Main St, Andover, MA",SINGLE_FAMILY,3.0,2.0,649000,7.805548e+05,131554.773544,20.270381,42.669825,-71.148779,http://ap.rdcpix.com/ba910a001a4d7113d0412e142...,https://www.realtor.com/realestateandhomes-det...
1,"8 Windemere Dr, Andover, MA",SINGLE_FAMILY,4.0,2.0,975000,9.529882e+05,-22011.826640,-2.257623,42.653420,-71.176641,http://ap.rdcpix.com/a0c22de9c3a829b2f5838f19d...,https://www.realtor.com/realestateandhomes-det...
9,"Chandler Rd, Andover, MA",SINGLE_FAMILY,5.0,4.0,2195000,1.902521e+06,-292478.671575,-13.324769,42.669371,-71.175533,http://ap.rdcpix.com/a47bd6e7ccd36002f1a8d0c6d...,https://www.realtor.com/realestateandhomes-det...
8,"Weeping Willow Way, Andover, MA",SINGLE_FAMILY,4.0,3.0,2495000,2.016483e+06,-478517.186861,-19.179046,42.656857,-71.140789,http://ap.rdcpix.com/e5bda82502168234896a81873...,https://www.realtor.com/realestateandhomes-det...
11,"72 Tewksbury St, Andover, MA",SINGLE_FAMILY,5.0,5.0,3495000,2.576954e+06,-918046.258885,-26.267418,42.621981,-71.166097,http://ap.rdcpix.com/cb784a45952e516f9df0e9199...,https://www.realtor.com/realestateandhomes-det...
12,"117 Bellevue Rd, Andover, MA",SINGLE_FAMILY,3.0,3.0,1799000,1.036414e+06,-762585.609382,-42.389417,42.637317,-71.209572,http://ap.rdcpix.com/c0c83b14971f6834c5fc35cb6...,https://www.realtor.com/realestateandhomes-det...


In [83]:
importances = redfin.print_feature_importances()
print(f"===\nFeature Importances ({len(importances)})\n===")
for importance in importances:
    print(importance)



===
Feature Importances (17)
===
('sqft', 0.7274734155889375)
('year_built', 0.09405112820651333)
('lot_sqft', 0.07487608805720712)
('full_baths', 0.03534335045935998)
('half_baths', 0.02423000564975376)
('parking_garage', 0.01992904868991012)
('city=Andover', 0.011295274912504162)
('city=In Town', 0.0052008392867180855)
('beds', 0.004972690516957235)
('hoa_fee', 0.0013761896223650016)
('city=South Andover', 0.000822142136835497)
('city=Shawsheen Village', 0.000185665139757321)
('stories', 0.00012651435020161523)
('city=Ballardvale', 0.00011764738297907314)
('city=West Andover', 0.0)
('state=MA', 0.0)
('style=SINGLE_FAMILY', 0.0)


In [84]:
# test_df.head()
print(set(result_df['style']))

{'SINGLE_FAMILY'}


In [85]:
result_df[RedfinModel.OUTPUT_COLUMNS].to_dict(orient='records')[0]

{'readable_address': '124  Lovejoy Rd, Andover, MA',
 'style': 'SINGLE_FAMILY',
 'beds': 4.0,
 'full_baths': 4.0,
 'list_price': 999900,
 'predicted': 1770671.8358958818,
 'diff': 770771.8358958818,
 'diff_percent': 77.08489207879606,
 'latitude': 42.646538,
 'longitude': -71.179352,
 'primary_photo': 'http://ap.rdcpix.com/ac67958e0fea7a9ea2824ad5419b6413l-m3043692644od-w480_h360_x2.webp?w=1080&q=75',
 'property_url': 'https://www.realtor.com/realestateandhomes-detail/4529411344'}